## Diet problem

A canteen has to plan the composition of the meals that it provides. A meal can be composed of the types of food indicated in the following table. Costs, in Euro per hg, and availabilities, in hg, are also indicated.

Food|Cost|Availability
----|----|------------
Bread|0.1|4
Milk|0.5|3
Eggs|0.12|1
Meat|0.9|2
Cake|1.3|2

A meal must contain at least the following amount of each nutrient

Nutrient|Minimal quantity
--------|----------------
Calories|600 cal
Proteins|50 g
Calcium|0.7 g

Each hg of each type of food contains to following amount of nutrients

Food|Calories|Proteins|Calcium
----|--------|--------|-------
Bread|30 cal|5 g|0.02 g
Milk|50 cal|15 g|0.15 g
Eggs|150 cal|30 g|0.05 g
Meat|180 cal|90 g|0.08 g
Cake|400 cal|70 g|0.01 g

Give a linear programming formulation for the problem of finding a diet (a meal) of minimum total cost which satisfies the minimum nutrient requirements.

Relevant sets:

$I$= set of types of foods

$J$= set of types of nutrients

$I$={Bread, Milk, Eggs...} when we refer to elements we use

$J$ = {Calories, Proteins, Calcium} we will use

**Decision variables**

$x_s$ = amount of food type $i \in I$

**Parameters**
From table:

$c_i$ = cost of food type $i \in I$

$q_i$ = available quantity of food type $i \in I$

From table 2:
$b_j$ = minimum quantity of nutrient $j \in J$

From table 3:

$a_{ij}$ = the amount of nutrient $j \in J$ for the food type $i \in I$

**Constraints**

Minimal quantity of nutrients in the solution

$$ \sum_{i \in I} e_{ij} x_i \geq b_j, \ \ \ ∀ g ∈ J $$

Availabilty of food type:

$$ x_i \leq q_i, \ \ \ ∀ i ∈ I $$

Nature of decision variables:

$$ x_i \geq 0, x_i \in I$$

$ x_1 = 100.1 $ amount of bread,
$ x_2 = 10.9 $ (amount of milk) and so on...

**Objective function**
$$\min \sum_{i \in I} c_i x_i$$

$$\min 0.1x_1 + 0.5x_2 0.12x_3 + 0.9x_4 + 1.3x_5$$

All together
$$\min \sum_{i \in I} c_i x_i$$

$$ \sum_{i \in I} e_{ij} x_i \geq b_j, \ \ \ ∀ g ∈ J $$

$$ x_i \leq q_i, \ \ \ ∀ i ∈ I $$
$$ x_i \geq 0, x_i \in I$$

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 51.7 MB/s eta 0:00:00


In [2]:
# We need to import the package mip
import mip

In [3]:
# Food
I = ["Bread", "Milk", "Eggs", "Meat", "Cake"]

# Nutrients
J = ["Calories", "Proteins", "Calcium"]

# Cost in Euro per hg of food
c = {"Bread": 0.1, "Milk": 0.5, "Eggs": 0.12, "Meat" :0.9, "Cake": 1.3}

# Availability per hg of food
q = {"Bread": 4, "Milk": 3, "Eggs": 1, "Meat" :2, "Cake": 2}

# minum nutrients
b = {"Calories":600, "Proteins":50, "Calcium":0.7}

# Nutrients per hf of food
a = {('Bread','Calories'):30,('Milk','Calories'):50,('Eggs','Calories'):150,('Meat','Calories'):180,('Cake','Calories'):400,
('Bread','Proteins'):5,('Milk','Proteins'):15,('Eggs','Proteins'):30,('Meat','Proteins'):90,('Cake','Proteins'):70,
('Bread','Calcium'):0.02,('Milk','Calcium'):0.15,('Eggs','Calcium'):0.05,('Meat','Calcium'):0.08,('Cake','Calcium'):0.01}

Now we create an empty model and add the variables

In [6]:
# Define a model
model = mip.Model()

#add_var ()

#one way: one by one
x1 = model.add_var(var_type = mip.CONTINUOUS) #by default x1>= 0
x2 = model.add_var()
x3 = model.add_var()
x4 = model.add_var()
x5 = model.add_var()

# Define variables: using list comprehension
x = [model.add_var(name = i,lb=0) for i in I] #lb is the lower bound (minimum value that the variable could be)

In [7]:
print(x)

[<mip.entities.Var object at 0x780bbdda6800>, <mip.entities.Var object at 0x780bbdda5660>, <mip.entities.Var object at 0x780bbdda6cb0>, <mip.entities.Var object at 0x780bbdda62f0>, <mip.entities.Var object at 0x780bbdda5900>]


Let's write the objective function: in the general case, it can be written as a sum over the set of models.

In [8]:
# Define the objective function

#one way: directly
#model.objective = mip.minimize(0.1*x1 + 0.5*x2 +....)

#this function xsum() receives as input a list and return the sum for the model to read it
model.objective = mip.minimize(mip.xsum([c[food]*x[i] for i, food in enumerate(I)]))

The constraints can be generated in loops:

In [10]:
# CONSTRAINTS

# Availability constraint
for i,food in enumerate(I):
  model.add_constr(x[i]<=q[food])

# Minum nutrients
for j in J:
  model.add_constr(mip.xsum([a[food, j]*x[i] for i, food in enumerate(I)]) >= b[j])

The model is complete. Let's solve it and print the optimal solution.

In [11]:
# Optimizing command
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [12]:
# Optimal objective function value
model.objective.x

3.37

In [13]:
# Printing the variables values
for i in model.vars:
  print(i.name)
  print(i.x)

var(0)
0.0
var(1)
0.0
var(2)
0.0
var(3)
0.0
var(4)
0.0
Bread
3.9999999999999996
Milk
3.0
Eggs
1.0
Meat
1.5000000000000002
Cake
0.0


## Oil blending problem

A refinery has to blend 4 types of oil to obtain 3 types of gasoline. The following table reports the available quantity of oil of each type (in barrels) and the respective unit cost (Euro per barrel)

Oil type|Cost|Availability
--------|----|------------
1|9|5000
2|7|2400
3|12|4000
4|6|1500


Blending constraints are to be taken into account, since each type of gasoline must contain at least a specific, predefined, quantity of each type of oil, as indicated in the next table. The unit revenue of each type of gasoline (Euro per barrel) is also indicated

Gasoline type|Requirements|Revenue
-------------|------------|-------
A|$\geq$ 20% of type 2| 12
A|$\leq$ 30% of type 3|12
B|$\geq$ 40% of type 3|18
C|$\leq$ 50% of type 2|10


In [ ]:
# Set of oil types
I =

# Set of gasoline types
J =

# Unit cost for oil of type i
c =

# Availability of oil type i
b =

# Price of gasoline of type j
r =

# Maximum quantity (percentage) of oil
q_max = {}
for i in I:
  for j in J:
    q_max[(str(i),j)] = 1
q_max[('2','A')] = 0.3
q_max[('1','C')] = 0.5

# Minimum quantity (percentage) of oil
q_min = {}
for i in I:
  for j in J:
    q_min[(str(i),j)] = 0
q_min[('1','A')] = 0.2
q_min[('2','B')] = 0.4

Now we create an empty model and add the variables

In [ ]:
# Define a model
model2 = mip.Model()

# Define variables
x = {}
y = {}

In [ ]:
# Define the objective function
model2.objective = mip.maximize()

In [ ]:
# CONSTRAINTS
# Availability constraint
for i in I:
  model2.add_constr()

# Conservation constraint
for j in J:
  model2.add_constr()

# Maximum quantity
for i in I:
  for j in J:
    model2.add_constr()

# Minimum quantity
for i in I:
  for j in J:
    model2.add_constr()

The model is complete. Let's solve it and print the optimal solution.

In [ ]:
# Optimizing command
model2.optimize()

In [ ]:
# Optimal objective function value
model2.objective.x

In [ ]:
# Printing the variables values
for i in model2.vars:
  print(i.name)
  print(i.x)
  print('-----')